In [2]:
import numpy as np
import gym
from collections import deque
import random
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim


#### 经验回放池：

In [ ]:
class ReplayBuffer:
    ''' 经验回放池 '''
    def __init__(self, capacity):
        self.buffer = deque(maxlen=capacity)        # 先进先出队列

    def add(self, state, action, reward, next_state, done):  
        self.buffer.append((state, action, reward, next_state, done))

    def sample(self, batch_size):  
        transitions = random.sample(self.buffer, batch_size)
        state, action, reward, next_state, done = zip(*transitions)
        return np.array(state), np.array(action), reward, np.array(next_state), done

    def size(self): 
        return len(self.buffer)

#### DQN:
1. DQN 算法概述
DQN 是一种深度强化学习算法，结合 Q-Learning 和 深度神经网络，用于处理高维状态空间（如图像或连续状态）。由 DeepMind 在 2013 和 2015 年提出，广泛应用于 Atari 游戏等任务。DQN 使用神经网络近似 Q 值函数 $Q(s, a; \theta)$，通过优化时序差分（TD）误差学习最优策略。

2. 数学原理
2.1 强化学习与马尔可夫决策过程 (MDP)
- **强化学习建模为 MDP $(S, A, P, R, \gamma)$**：
状态 $S$：RollingBall 的 $[x, y, v_x, v_y]$。
动作 $A$：25 个离散动作（Hashposotion.py）。
转移概率 $P(s' | s, a)$：由 GridWorld.py 物理规则隐式定义。
奖励 $R(s, a, s')$：-2.0（每步）、-10.0（撞墙）、+300.0（目标）。
折扣因子 $\gamma$：通常 0.99。

- **目标：最大化累积折扣奖励**：
$$
J(\pi) = \mathbb{E}{\pi} \left[ \sum{t=0}^\infty \gamma^t R(s_t, a_t, s_{t+1}) \right]
$$
平稳分布 $d_\pi(s)$，满足 $d_\pi^T P_\pi = d_\pi^T$，描述策略 $\pi$ 下的长期状态访问概率。

2.2 Q-Learning 与 Bellman 方程
Q 值函数 $Q^\pi(s, a)$ 表示状态 $s$ 下执行动作 $a$，然后按策略 $\pi$ 行动的预期累积奖励：
$$Q^\pi(s, a) = \mathbb{E}{\pi} \left[ R(s, a, s') + \gamma \sum{a'} \pi(a'|s') Q^\pi(s', a') \right]$$
最优 Q 值函数 $Q^*(s, a)$ 满足 Bellman 最优方程（《Chapter 8》, Box 7.5）：
$$Q^(s, a) = \mathbb{E} \left[ R(s, a, s') + \gamma \max_{a'} Q^(s', a') \right]$$
Q-Learning 更新：
$$Q(s, a) \leftarrow Q(s, a) + \alpha \left[ R(s, a, s') + \gamma \max_{a'} Q(s', a') - Q(s, a) \right]$$
RollingBall 的连续状态使 Q 表格不可行，需函数逼近。
2.3 DQN 的函数逼近
DQN 使用神经网络 $Q(s, a; \theta)$ 近似 $Q^*(s, a)$。

表格法：存储所有 $Q(s, a)$，对连续状态不可行。
函数逼近：用参数 $\theta$ 表示 $Q(s, a; \theta)$，存储效率高，泛化能力强。

RollingBall 配置：

输入：4 维状态 $[x, y, v_x, v_y]$。
输出：25 维 Q 值向量。
网络：全连接网络（2-3 层，128 神经元，ReLU 激活）。

DQN 优化 Bellman 误差：
$$J(\theta) = \mathbb{E} \left[ \left( R + \gamma \max_{a'} Q(S', a'; \theta) - Q(S, A; \theta) \right)^2 \right]$$
直接优化不稳定，需以下技术。
3. DQN 的关键技术及数学推导
3.1 经验回放（Experience Replay）
问题：经验 $(s, a, r, s', \text{done})$ 具有时间相关性，违背神经网络训练的 i.i.d. 假设。
解决方案：回放池存储经验，随机采样批量数据。强调均匀采样确保状态-动作对 $(S, A)$ 近似均匀分布。
数学原理：

回放池 $\mathcal{D}$ 存储 $(s, a, r, s', \text{done})$。
采样 $N$ 个经验，损失为：

$$L(\theta) = \frac{1}{N} \sum_{i=1}^N \left[ r_i + \gamma (1 - \text{done}i) \max{a'} Q(s'_i, a'; \theta) - Q(s_i, a_i; \theta) \right]^2$$

3.2 目标网络（Target Network）
问题：目标值 $r + \gamma \max_{a'} Q(s', a'; \theta)$ 随 $\theta$ 变化，训练不稳定。
解决方案：目标网络 $Q(s, a; \theta^-)$，参数 $\theta^-$ 定期复制。《Chapter 8》（Section 8.4.1）描述固定 $\theta^-$ 简化梯度计算：
$$L(\theta) = \mathbb{E} \left[ \left( r + \gamma \max_{a'} Q(s', a'; \theta^-) - Q(s, a; \theta) \right)^2 \right]$$
数学推导：

目标值：

$$y_i = r_i + \gamma (1 - \text{done}i) \max{a'} Q(s'_i, a'; \theta^-)$$

梯度：

$$\nabla_\theta L(\theta) = -\frac{2}{N} \sum_{i=1}^N \left[ y_i - Q(s_i, a_i; \theta) \right] \nabla_\theta Q(s_i, a_i; \theta)$$

每 $k$ 步（如 100），$\theta^- \leftarrow \theta$。


3.3 ε-贪婪策略
问题：需平衡探索与利用。
解决方案：以概率 $\epsilon$ 随机选择动作，否则选择 $\arg\max_a Q(s, a; \theta)$。《Chapter 8》（Box 8.1）提到探索策略（如 ε-贪婪）确保平稳分布唯一。
数学原理：

策略：

$$\pi(a|s) = \begin{cases}\frac{\epsilon}{|A|} + (1 - \epsilon) \cdot \mathbb{I}[a = \arg\max_{a'} Q(s, a'; \theta)] & \text{if } a = \arg\max_{a'} Q(s, a'; \theta) \\frac{\epsilon}{|A|} & \text{otherwise}\end{cases}$$

衰减：

$$\epsilon_t = \epsilon_{\text{end}} + (\epsilon_{\text{start}} - \epsilon_{\text{end}}) \exp(-t / \tau)$$
与 RollingBall 的结合：

env.action_space.sample() 实现随机动作。
衰减 $\epsilon$（如 1.0 到 0.1）鼓励目标探索。

4. DQN 算法流程
综合《Chapter 8》（Algorithm 8.3）和 hrl.boyuai.com：

初始化：

在线网络 $Q(s, a; \theta)$，目标网络 $Q(s, a; \theta^-)$，$\theta^- \leftarrow \theta$。
回放池 $\mathcal{D}$，容量 capacity。
超参数：学习率 $\alpha$，折扣因子 $\gamma$，$\epsilon$ 参数。

交互：

按 ε-贪婪选择 $a$。
执行 $a$，获取 $r, s', \text{done}$。
存储 $(s, a, r, s', \text{done})$。

优化：

采样批量经验。

目标值：
$$y_i = r_i + \gamma (1 - \text{done}i) \max{a'} Q(s'_i, a'; \theta^-)$$

损失：
$$L(\theta) = \frac{1}{N} \sum_{i=1}^N \left[ y_i - Q(s_i, a_i; \theta) \right]^2$$

更新：

每 $k$ 步，$\theta^- \leftarrow \theta$。
衰减 $\epsilon_t$。

5. 局限与改进

Q 值过估计：

$\max_{a'} Q(s', a'; \theta^-)$ 可能高估。《Chapter 10》（Section 10.2）提到基线减少方差，DQN 用 Double DQN：
$$y_i = r_i + \gamma (1 - \text{done}_i) Q(s'i, \arg\max{a'} Q(s'_i, a'; \theta); \theta^-)$$

稀疏奖励：RollingBall 的 +300.0 奖励稀疏，需奖励整形。

离散动作：DQN 限于离散动作

In [ ]:
class Q_Net(torch.nn.Module):
    ''' Q 网络是一个两层 MLP, 用于 DQN 和 Double DQN '''
    def __init__(self, input_dim, hidden_dim, output_dim):
        super().__init__()
        self.fc1 = torch.nn.Linear(input_dim, hidden_dim)
        self.fc2 = torch.nn.Linear(hidden_dim, hidden_dim)
        self.fc3 = torch.nn.Linear(hidden_dim, output_dim)
        # 初始化权重
        self._init_weights()
    
    def _init_weights(self):
        """使用Kaiming初始化权重，适合激活函数为Relu"""
        for module in self.modules():
            if isinstance(module, nn.Linear):
                # 使用 Kaiming 均匀初始化，指定 nonlinearity='relu'
                nn.init.kaiming_uniform_(module.weight, nonlinearity='relu')
                # 偏置置零
                if module.bias is not None:
                    nn.init.zeros_(module.bias)
                    
    def forward(self, x):
        x = F.relu(self.fc1(x)) 
        x = F.relu(self.fc2(x))
        return self.fc3(x)

1. self.bn1 = nn.BatchNorm1d(hidden_dim) 是定义一个一维批归一化（Batch Normalization）层的操作。
- **核心功能**针对全连接层（Linear）或一维卷积层（Conv1D）的输出.对每个隐藏层神经元的输出进行标准化处理：
    $$
    \hat{x}_i = \frac{x_i - \mu_B}{\sqrt{\sigma_B^2 + \epsilon}} \quad \text{(标准化)}
    $$
    $$
    y_i = \gamma \hat{x}_i + \beta \quad \text{(可学习缩放和平移)}
    $$
  其中：
$μ_B$是该神经元在当前batch上的均值,$σ_B^2$是方差,γ(weight)和β(bias)是可学习的参数。

- **在DQN中的具体作用:**

    正向传播时：标准化过程计算当前batch中每个神经元的均值和方差，对128个神经元的输出独立标准化，通过γ和β保留网络的表达能力

    反向传播时：自动更新当前batch的$μ_B$和$σ_B^2$、可学习参数γ和β、全局统计量（用于推理的running_mean和running_var）

- **数据流维度变化：**

    输入: [batch_size, 4] 
    
    fc1: [batch_size, 128] 

    bn1: [batch_size, 128] (标准化后)

    fc2: [batch_size, 128]

    输出: [batch_size, 25]
    通过这种设计，BatchNorm1d能有效提升DQN在RollingBall环境中的训练稳定性和收敛速度。

---

In [ ]:
class DQN(torch.nn.Module):
    ''' DQN算法 '''
    def __init__(self, state_dim, hidden_dim, action_dim, action_range, lr, gamma, epsilon, device, tau=0.001, seed=None):
        super().__init__()
        self.action_dim = action_dim
        self.state_dim = state_dim
        self.hidden_dim = hidden_dim
        self.action_range = action_range        # action 取值范围
        self.gamma = gamma                      # 折扣因子
        self.epsilon = epsilon                  # epsilon-greedy
        self.tau = tau      # 目标网络更新频率
        self.count = 0                          # Q_Net 更新计数
        self.rng = np.random.RandomState(seed)  # agent 使用的随机数生成器
        self.device = device                
        
        # Q 网络
        self.q_net = Q_Net(state_dim, hidden_dim, action_range).to(device)  
        # 目标网络
        self.target_q_net = Q_Net(state_dim, hidden_dim, action_range).to(device)
        # 使用Adam优化器
        self.optimizer = torch.optim.Adam(self.q_net.parameters(), lr=lr)
        
    def max_q_value_of_given_state(self, state):
        state = torch.tensor(state, dtype=torch.float).to(self.device)
        return self.q_net(state).max().item()
        
    def take_action(self, state):  
        ''' 按照 epsilon-greedy 策略采样动作 '''
        if self.rng.random() < self.epsilon:
            action = self.rng.randint(self.action_range)
        else:
            state = torch.tensor(state, dtype=torch.float).to(self.device)
            action = self.q_net(state).argmax().item()
        return action

    def update(self, transition_dict):
        states = torch.tensor(transition_dict['states'], dtype=torch.float).to(self.device)                             # (bsz, state_dim)
        next_states = torch.tensor(transition_dict['next_states'], dtype=torch.float).to(self.device)                   # (bsz, state_dim)
        actions = torch.tensor(transition_dict['actions'], dtype=torch.int64).view(-1, 1).to(self.device)               # (bsz, act_dim)
        rewards = torch.tensor(transition_dict['rewards'], dtype=torch.float).view(-1, 1).to(self.device).squeeze()     # (bsz, )
        dones = torch.tensor(transition_dict['dones'], dtype=torch.float).view(-1, 1).to(self.device).squeeze()         # (bsz, )

        q_values = self.q_net(states).gather(dim=1, index=actions).squeeze()                # (bsz, )
        max_next_q_values = self.target_q_net(next_states).max(axis=1)[0]                   # (bsz, )
        q_targets = rewards + self.gamma * max_next_q_values * (1 - dones)                  # (bsz, )

        dqn_loss = torch.mean(F.mse_loss(q_values, q_targets))  
        self.optimizer.zero_grad()                                                         
        dqn_loss.backward() 
        self.optimizer.step()
        
        # 软更新目标网络参数
        for target_param, q_param in zip(self.target_q_net.parameters(), self.q_net.parameters()):
            target_param.data.copy_(self.tau * q_param.data + (1.0 - self.tau) * target_param.data)


#### DoubleDQN:

In [ ]:
class DoubleDQN(DQN):
    ''' Double DQN算法 '''
    def __init__(self, state_dim, hidden_dim, action_dim, action_range, lr, gamma, epsilon, device, tau=0.001, seed=None):
        super().__init__(state_dim, hidden_dim, action_dim, action_range, lr, gamma, epsilon, device, tau, seed)
    
    def update(self, transition_dict):
        states = torch.tensor(transition_dict['states'], dtype=torch.float).to(self.device)                             # (bsz, state_dim)
        next_states = torch.tensor(transition_dict['next_states'], dtype=torch.float).to(self.device)                   # (bsz, state_dim)
        actions = torch.tensor(transition_dict['actions'], dtype=torch.int64).view(-1, 1).to(self.device)               # (bsz, act_dim)
        rewards = torch.tensor(transition_dict['rewards'], dtype=torch.float).view(-1, 1).to(self.device).squeeze()     # (bsz, )
        dones = torch.tensor(transition_dict['dones'], dtype=torch.float).view(-1, 1).to(self.device).squeeze()         # (bsz, )
        # Double DQN：主网络选择动作，目标网络估计Q值
        q_values = self.q_net(states).gather(dim=1, index=actions).squeeze()                # (bsz, )
        # 使用Q网络估计最优动作（[0]取最优值，[1]取最优值的索引）
        max_actions_index = self.q_net(next_states).max(axis=1)[1]
        # 由目标网络计算Q值
        max_next_q_values = self.target_q_net(next_states).gather(dim=1, index = max_actions_index.unsqueeze(1)).squeeze()                   # (bsz, )
        q_targets = rewards + self.gamma * max_next_q_values * (1 - dones)                  # (bsz, )

        dqn_loss = torch.mean(F.mse_loss(q_values, q_targets))  
        self.optimizer.zero_grad()                                                         
        dqn_loss.backward() 
        self.optimizer.step()
        
        # 软更新目标网络参数
        for target_param, q_param in zip(self.target_q_net.parameters(), self.q_net.parameters()):
            target_param.data.copy_(self.tau * q_param.data + (1.0 - self.tau) * target_param.data)